# Yonsei Major parser


In [5]:
# !pip install unstructured

In [6]:
from unstructured.partition.pdf import partition_pdf

In [7]:
pages_to_use = list(range(132, 240))

elements = partition_pdf(
    'data/yonsei/raw/yonsei_raw.pdf',
    strategy="fast",            # 디지털 PDF면 fast부터 시도
    include_page_breaks=True,
    pages=pages_to_use,
    infer_table_structure=True, # 표 구조 추론 시도
)

In [8]:
from unstructured.documents.elements import Title, NarrativeText, Table

print(len(elements))          # 전체 조각 개수
print(elements[0])            # 한 개 찍어보기

e = elements[0]

print(type(e))              # <class 'unstructured.documents.elements.Title'>
print(e.__class__.__name__) # "Title"
print(e.text)               # 실제 텍스트
print(e.metadata.page_number)

96582
대학별 교과과정
<class 'unstructured.documents.elements.Title'>
Title
대학별 교과과정
1


In [9]:
# 원하는 타입만 필터링
# 1. 본문만
from unstructured.documents.elements import NarrativeText

narratives = [e for e in elements if isinstance(e, NarrativeText)]
all_text = "\n\n".join(e.text for e in narratives)

In [10]:
# 2. 제목만
from unstructured.documents.elements import Title

titles = [e for e in elements if isinstance(e, Title)]
for t in titles:
    print(f"[p{t.metadata.page_number}] {t.text}")

[p1] 대학별 교과과정
[p1] 융합대학
[p2] 부전공
[p2] 국어국문학전공
[p2] 중어중문학전공
[p2] 영어영문학전공
[p2] 독어독문학전공
[p2] 불어불문학전공
[p2] 노어노문학전공
[p2] 사학전공
[p2] 철학전공
[p2] 문헌정보학전공
[p2] 심리학전공
[p2] 경제학부
[p2] 부전공 및 마이크로전공과목 일람표
[p2] 문과대학
[p2] 우리말연구의첫걸음
[p2] 문학이란무엇인가
[p2] 한국고전문학의이해
[p2] 과목 필
[p2] 독어독문학의이해 독어연습 독어독문학 과목 중 학점 이상 선택 총 학점 이상 이수
[p2] 3 독어학의이해 이상 과목 학점 필수 이수
[p2] 전공필수 교과목 프랑스문학의이해 프랑스어학의이해 프랑스어권명저의이해 프랑스산문강독 ( 랑스어작문과표현 프랑스어회화 이상 이수
[p2] 프 학점과 전공선택 교과목 학점 이상을 포함하여 총 학점
[p2] 중급러시아어연습
[p2] 고급러시아어
[p2] 21 ( 필수 학점 전공 학점 전공기초 학점 전공필
[p2] . 학년도 입학생 심리학개론 심리학의실험연구방법 심리통계
[p2] 상경대학
[p3] 을 추가로 이수해야 한다.
[p3] ② 경제학 부전공 학점이수에 세부사항에 관하여는 경제학부 사무실로 문의 바람 1)
[p3] 학번 이후 부전공
[p3] 학점
[p3] 미시경제원론
[p3] 미시경제학
[p3] 학점
[p3] 학번 부전공 경제수학 (3), ~2005
[p3] 통계학입문
[p3] 학점
[p3] 거
[p3] 응용통계학과
[p3] 학번부터 경영대학 교차인정
[p3] 학번부터 프로그래밍 파이썬프로그래밍 두 과목을 모두 이수한 경우 한 과목은 전공선
[p3] 학번부터 부전공
[p3] ) 학점 주의 대학교양 통계학입문 이수 요건 아님
[p3] , R
[p3] 전공선택 학점
[p3] 전공필수 학점 수리통계학
[p3] 전공선택 학점
[p3] 전공선택 학점
[p3] 경영대학
[p3] 경영학과
[p3] 조직행동론
[p3] 오퍼레이션관리 이상 과목 학점 필수 이수
[p3] 재무관리
[p3]

In [11]:
# 3. 테이블만
from unstructured.documents.elements import Table

tables = [e for e in elements if isinstance(e, Table)]

print("표 개수:", len(tables))
for i, tbl in enumerate(tables[:3], start=1):
    print(f"=== Table {i} (page {tbl.metadata.page_number}) ===")
    print(tbl.text[:500])   # 혹은 tbl.text 전체

표 개수: 0


In [12]:
# 페이지 단위로 묶어서
from collections import defaultdict

by_page = defaultdict(list)
for e in elements:
    page = e.metadata.page_number
    by_page[page].append(e)

# 페이지별 순서대로 텍스트 합치기
page_texts = {
    page: "\n".join(el.text for el in els if el.text)
    for page, els in by_page.items()
}

# 예: 132페이지 내용 확인
print(page_texts.get(132, "no data"))

세상의 틀을 깨는 도전으로 선한 영향력을 나누는 인재
이 과 대 학
이과대학은 자연과학 연구 및 교육의 선구자적 역할을 담당하면서 한국의 자연과학 발전을 이끌어 왔다 해방 후 대학교 이학원으로 인가되었고 부로 분리되었다가 와 명실공히 현대적 과학교육과 연구 전당으로서의 면모를 갖추었다 이후 연구 및 교육 활동이 급격히 증가됨에 따라 년 월에 과학원 주년 해인 템대학이 분리되었다.
1915
년 월 조선기독교대학의 수물과로 시작하여
3
1917
,
4
년 월 연희전문학교의 수물과로 인가된 이래 현대 년 월에는 연희 년 월에는 그 규모가 확장되어 이학부와 공학 평 으로 옮겨 2000 년 생명시스
.
1946
8
8 1984
, 1950
년에는 이공대학으로 승격되었으며
, 1961 년에 이과대학과 공과대학으로 분리되었다 이과대학은
. . 2015
2
(6,560 )
1977
년 월에 현 과학관
(4,209 )
7
,
평 을 준공하였으며 창립
년에 과학관을 증축
100
평 하였다
(1,260 )
. 2008
현재 이과대학은 수학 물리학 화학 지구시스템과학 천문우주학 대기과학의 개 학과로 구성되어 있고 대형연구원인 , 100 여 명의 교수진이 활발한 연 , , 여 명의 동문들은 학계 산업계 공공 연구기관 정부기관 등
,
,
,
,
6
,
6
. 자연과학연구원 아래 개의 부설연구소를 가지고 있다 이과대학은 현재 세계적으로 우수한 구와 교육 활동을 펼치고 있다 이과대학에서 배출된 약 국내외 다양한 분야에서 선도적인 역할을 담당하고 있다. ,
,
.
18,000
창의적 연구진흥사업
사업 우수연구센터
이과대학은 (IBS)
기초과학연 . 1995 년 구원 년 환경지질연구정보센 년 초미세표면과학연구센터 ( 교육부로부터 대학원중심대학 연구중심대학 으로 지정되었고 년에는 교육부 단계 1 터 년 자외선우주망원경연구단과 인공지질이중분자막연구단이 각각 선정되었다 두뇌한국 사업에서 물리학분야와 수학분야가 과학기술분야에서 나머지 모든 전공이 핵심 분야에서 선정되었다 또한 2000 년에는 초고속